In [1]:
import pandas as pd
import numpy as np
import statsmodels as sm
import matplotlib.pyplot as plt
import seaborn as sns
import math
import datetime as dt

# Data Cleaning

First, we format dates for pandas and then we merge on the dates and cusip (ticker but stronger).

In [227]:
#Jordan: to run u need to change these paths; dont delete them just comment them out; im not sure if the sytnax is the same on a mac
crsp = pd.read_csv('C:\\Users\\nfischer\\Downloads\\CRSPprices2.csv')
compustat = pd.read_csv('C:\\Users\\nfischer\\Downloads\\CompustatRatings.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [233]:
crsp

,PERMNO,date,TICKER,COMNAM,cusip,NWPERM,PRC,VOL,RET
0,10001,20100104,EGAS,ENERGY INC,36720410,NaN,10.25000,18500.0,-0.004854
1,10001,20100105,EGAS,ENERGY INC,36720410,NaN,10.19000,23200.0,-0.005854
2,10001,20100106,EGAS,ENERGY INC,36720410,NaN,10.31000,18700.0,0.011776
3,10001,20100107,EGAS,ENERGY INC,36720410,NaN,9.96000,29200.0,-0.033948
4,10001,20100108,EGAS,ENERGY INC,36720410,NaN,10.34000,25100.0,0.038153
...,...,...,...,...,...,...,...,...,...
14024949,93436,20171222,TSLA,TESLA INC,88160R10,NaN,325.20001,4215911.0,-0.019478
14024950,93436,20171226,TSLA,TESLA INC,88160R10,NaN,317.29001,4378413.0,-0.024324
14024951,93436,20171227,TSLA,TESLA INC,88160R10,NaN,311.64001,4712111.0,-0.017807
14024952,93436,20171228,TSLA,TESLA INC,88160R10,NaN,315.35999,4316358.0,0.011937


In [237]:
compustat

,gvkey,splticrm,datadate,conm,tic,cusip,date,NCUSIP,TICKER
0,1003,NaN,20100131,A.A. IMPORTING CO INC,ANTQ,00035410,2010-01-31,000354100,ANTQ
1,1003,NaN,20100228,A.A. IMPORTING CO INC,ANTQ,00035410,2010-02-28,000354100,ANTQ
2,1003,NaN,20100331,A.A. IMPORTING CO INC,ANTQ,00035410,2010-03-31,000354100,ANTQ
3,1003,NaN,20100430,A.A. IMPORTING CO INC,ANTQ,00035410,2010-04-30,000354100,ANTQ
4,1003,NaN,20100531,A.A. IMPORTING CO INC,ANTQ,00035410,2010-05-31,000354100,ANTQ
...,...,...,...,...,...,...,...,...,...
774284,316056,BBB-,20161031,ALLEGION PLC,ALLE,G0176J10,2016-10-31,G0176J109,ALLE
774285,316056,BBB-,20161130,ALLEGION PLC,ALLE,G0176J10,2016-11-30,G0176J109,ALLE
774286,316056,BBB-,20161231,ALLEGION PLC,ALLE,G0176J10,2016-12-31,G0176J109,ALLE
774287,316056,BBB-,20170131,ALLEGION PLC,ALLE,G0176J10,2017-01-31,G0176J109,ALLE


In [236]:
#everything involving crsp is memory intensive; make sure u have ~3gb ram available to rerun this
compustat['date'] = pd.to_datetime(compustat['datadate'], format='%Y%m%d')
crsp['date'] = pd.to_datetime(crsp['date'], format='%Y%m%d')

In [238]:
#cut to same date range; compustat most restrictive with beginning date as below
crsp = crsp.loc[crsp['date'] >= dt.datetime(2010,1,31)]

In [260]:
crsp = crsp.rename(columns={"CUSIP": "cusip", "tic":"TICKER"})

In [261]:
#crsp has daily data and compustat has last day of the month data; we want to convert this to price data on the last tradeable day each month;
dates = pd.date_range(start='1/31/2010', end='2/28/2017', freq='BM') #BM = business month; last tradeable day of each month
crsp = crsp[crsp['date'].isin(dates)]

In [262]:
#compustat has last day of each month; we force to last calendar day of each month for merge
crsp['date'] = pd.to_datetime(crsp['date'], format='%Y%m')+pd.tseries.offsets.MonthEnd(1)

In [265]:
compustat['cusip'] = compustat['cusip'].apply(lambda x: str(x)[:-1])

In [266]:
compustat['TICKER'] = compustat['tic']

In [267]:
crsp['conm'] = crsp['COMNAM']

In [263]:
crsp

,PERMNO,date,TICKER,COMNAM,cusip,NWPERM,PRC,VOL,RET,conm
60,10001,2010-05-31,EGAS,ENERGY INC,36720410,NaN,10.17000,8300.0,0.004792,ENERGY INC
81,10001,2010-06-30,EGAS,ENERGY INC,36720410,NaN,11.39000,23400.0,-0.011285,ENERGY INC
166,10001,2010-10-31,EGAS,GAS NATURAL INC,36720410,NaN,10.37100,6000.0,-0.002798,GAS NATURAL INC
229,10001,2011-01-31,EGAS,GAS NATURAL INC,36720410,NaN,10.04000,18400.0,-0.000985,GAS NATURAL INC
251,10001,2011-02-28,EGAS,GAS NATURAL INC,36720410,NaN,10.52000,17600.0,-0.000950,GAS NATURAL INC
...,...,...,...,...,...,...,...,...,...,...
14024553,93436,2016-07-31,TSLA,TESLA MOTORS INC,88160R10,NaN,223.23000,2789002.0,0.000852,TESLA MOTORS INC
14024618,93436,2016-10-31,TSLA,TESLA MOTORS INC,88160R10,NaN,212.00999,3276548.0,0.003170,TESLA MOTORS INC
14024639,93436,2016-11-30,TSLA,TESLA MOTORS INC,88160R10,NaN,204.03000,2586273.0,0.016592,TESLA MOTORS INC
14024660,93436,2016-12-31,TSLA,TESLA MOTORS INC,88160R10,NaN,197.73000,4709373.0,-0.011202,TESLA MOTORS INC


In [264]:
compustat

,gvkey,splticrm,datadate,conm,tic,cusip,date,NCUSIP,TICKER
0,1003,NaN,20100131,A.A. IMPORTING CO INC,ANTQ,00035410,2010-01-31,000354100,ANTQ
1,1003,NaN,20100228,A.A. IMPORTING CO INC,ANTQ,00035410,2010-02-28,000354100,ANTQ
2,1003,NaN,20100331,A.A. IMPORTING CO INC,ANTQ,00035410,2010-03-31,000354100,ANTQ
3,1003,NaN,20100430,A.A. IMPORTING CO INC,ANTQ,00035410,2010-04-30,000354100,ANTQ
4,1003,NaN,20100531,A.A. IMPORTING CO INC,ANTQ,00035410,2010-05-31,000354100,ANTQ
...,...,...,...,...,...,...,...,...,...
774284,316056,BBB-,20161031,ALLEGION PLC,ALLE,G0176J10,2016-10-31,G0176J109,ALLE
774285,316056,BBB-,20161130,ALLEGION PLC,ALLE,G0176J10,2016-11-30,G0176J109,ALLE
774286,316056,BBB-,20161231,ALLEGION PLC,ALLE,G0176J10,2016-12-31,G0176J109,ALLE
774287,316056,BBB-,20170131,ALLEGION PLC,ALLE,G0176J10,2017-01-31,G0176J109,ALLE


In [268]:
#merge by date and ticker on ratings; for our purposes, we dont need the CCM linking table, ticker is sufficient
df = pd.merge(compustat, crsp, how='left', on=['date', 'TICKER'])

In [272]:
del df['conm_x'], df['conm_y']

In [278]:
df

,gvkey,splticrm,datadate,tic,cusip_x,date,NCUSIP,TICKER,PERMNO,COMNAM,cusip_y,NWPERM,PRC,VOL,RET
0,1003,NaN,20100131,ANTQ,0003541,2010-01-31,000354100,ANTQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1003,NaN,20100228,ANTQ,0003541,2010-02-28,000354100,ANTQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1003,NaN,20100331,ANTQ,0003541,2010-03-31,000354100,ANTQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1003,NaN,20100430,ANTQ,0003541,2010-04-30,000354100,ANTQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1003,NaN,20100531,ANTQ,0003541,2010-05-31,000354100,ANTQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832964,316056,BBB-,20161031,ALLE,G0176J1,2016-10-31,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,71.22,573928.0,-0.000561
832965,316056,BBB-,20161130,ALLE,G0176J1,2016-11-30,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,68.91,1014660.0,0.009522
832966,316056,BBB-,20161231,ALLE,G0176J1,2016-12-31,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,63.84,1601109.0,0.003931
832967,316056,BBB-,20170131,ALLE,G0176J1,2017-01-31,G0176J109,ALLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [280]:
df.to_csv('C:\\Users\\nfischer\\Downloads\\CleanSOSC.csv')